In [6]:
import gym
import random
import numpy as np
import tensorflow as tf
import time
from gym.envs.registration import register
from IPython.display import clear_output
from collections import deque
print("Using OpenAI Gym:", gym.__version__)
print("Using Tensorflow:", tf.__version__)

Using OpenAI Gym: 0.10.5
Using Tensorflow: 1.9.0


In [2]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [13]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.replay_buffer = deque(maxlen=1000)
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[None])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.target_in = tf.placeholder(tf.float32, shape=[None])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience, batch_size=50):
        self.replay_buffer.append(experience)
        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(experience, *samples))
#         state, action, next_state, reward, done = ([exp] for exp in experience)
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next, axis=1)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        
agent = QNAgent(env)

Action size: 4
State size: 16


In [15]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        with tf.variable_scope("q_table", reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)

s: 15 a: 2
Episode: 99, Total reward: 73.0, eps: 0.13397967485796175
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 0.2915219   0.4591379   0.2689546   0.31623498]
 [ 0.31743488 -0.3748523   0.04215977  0.00409675]
 [ 0.1477144   0.48421127  0.04322524  0.31961307]
 [ 0.00712917  0.08716363  0.0486892   0.02223596]
 [ 0.31576723  0.48447582 -0.51515913  0.311779  ]
 [ 0.0684554   0.04598576  0.05356741 -0.01328123]
 [-0.42640948  0.5349467  -0.41976473  0.30681062]
 [ 0.25691265  0.30407184 -0.5459194   0.25798124]
 [ 0.34918264 -0.38702574  0.41161922  0.34177908]
 [ 0.35890004  0.27314752  0.437125   -0.5167427 ]
 [ 0.2928187   0.56286657 -0.51294297  0.39509773]
 [-0.36676133  0.50335765 -0.16498888 -0.16828948]
 [ 0.01138669  0.49019372 -0.12844104  0.17367804]
 [-0.53203475  0.01639959  0.24097243 -0.0812238 ]
 [ 0.08020027  0.47967038  0.49306536  0.0293826 ]
 [ 0.00829726  0.12029958 -0.30489913 -0.42049965]]
